In [16]:
# 读取nc文件
import netCDF4 as nc

# 地图
import cartopy.crs as ccrs

# 进行数组计算
import numpy as np

# 进行绘图时使用
import matplotlib.pyplot as plt

# 基础知识
## NetCDF 数据
- netcdf存储的数据就是一个多自变量的单值函数。用公式来说就是f(x,y,z,...)=value
- 函数的自变量x,y,z等在netcdf中叫做维(dimension)或坐标轴(axis)
- 函数值value在netcdf中叫做变量(Variables)
- 而自变量和函数值在物理学上的一些性质，比如计量单位(量纲)、物理学名称等等在netcdf中就叫属性(Attributes)

## 参考教程笔记
[NetCDF(nc)读写与格式转换介绍](https://blog.csdn.net/weixin_44785184/article/details/128734138?spm=1001.2101.3001.6650.4&utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7ECTRLIST%7ERate-4-128734138-blog-103492575.235%5Ev38%5Epc_relevant_sort_base3&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7ECTRLIST%7ERate-4-128734138-blog-103492575.235%5Ev38%5Epc_relevant_sort_base3&utm_relevant_index=9)575

## 新建、打开和关闭文件
- nc.Dataset(filename, mode)
    - filename 需要用双引号括起来
- mode="w" 创建新文件写，已经存在的文件会被覆盖掉
- mode="r" 只读，文件必须存在，此为默认的 mode
- mode="r+" 读写，文件必须存在
- mode="a" 打开已经存在的文件进行读写，如果不存在则创建一个新文件读写

In [79]:
fname = "test.nc"
f = nc.Dataset(fname, mode="a", format="NETCDF4")  # 打开nc文件并添加数据
#f = nc.Dataset(fname, mode="w", format="NETCDF4")  # 新建nc文件 nc.Dataset
#f = nc.Dataset(fname, mode="r", format="NETCDF4")  # 打开nc文件
#f = nc.Dataset(fname, mode="r+", format="NETCDF4") # 打开nc文件并添加数据
#简写： f=nc.Dataset("test.nc","a","NETCDF4")

#f.close() # 关闭文件

## 维度创建和读取
### 创建维度

time  = f.createDimension("time", None)  # unlimited维度
lat   = f.createDimension("lat", 73)     # 维度名lat，维度大小73
lon   = f.createDimension("lon", 144)
time.isunlimited() #判断维度是否是unlimited维度

### 读取维度
- print('1. Dimensions: ',f.dimensions) #维度 dimensions
- print('2. Variables: ',f.dimensions.keys()) #变量 variables
- print('3. Global attribute names: ',f.ncattrs())#全局属性ncattrs()，获取name getncattr获取value
- print('4. Global attribute values: ',[dataset.getncattr(i) for i in dataset.ncattrs()])

- .createDimension(DimensionName,维度大小)
    - 0或者None表示不限制维度大小
- .len()返回维度大小
- .isunlimited()判断当前维度是否无限制

In [80]:
level = f.createDimension("level", None)
time = f.createDimension("time", None)
lat = f.createDimension("lat", 73)
lon = f.createDimension("lon", 144)

In [81]:
# 查看维度大小
nlats = len(f.dimensions["lat"])
nlons = len(f.dimensions["lon"])
print(nlats, nlons, lon.isunlimited())

73 144 False


## 变量的创建、读取和存储
### 创建变量
- .createVariable(变量名，变量数据类型，变量所在纬度)
- 数据类型:
    - "f4"：32位浮点数
    - “f8”: 64位浮点数
    - “i4”: 32位有符号整数
    - “i2”：16位有符号整数
    - “i8”：64位有符号整数
    - “i1”：8位有符号整数
    - “u1”：8位无符号整数

In [82]:
# 变量名"level"，数据类型"i4"，维度名("level",)
times = f.createVariable("time","f8",("time",))
level = f.createVariable("level", "i4", ("level",))
latitudes  = f.createVariable("lat","f4",("lat",))
longitudes = f.createVariable("lon","f4",("lon",))
# 变量属性
latitudes.units  = "degrees north"   # 添加变量属性
longitudes.units = "degrees east"
times.units      = "hours since 0001-01-01 00:00:00.0"
times.calendar   = "gregorian"
level.units = "hPa"
temp.units = "K"

In [83]:
# 多维数据的维度排列一般位时间，纬度，经度（时间，垂直层，纬度，经度）
temp = f.createVariable("temp","f4",("time","level","lat","lon",))
temp.units      = "K"                 # 变量单位
temp.decription = "Air temperature"   # 变量说明
temp._Fillvalue  = -999               # 缺省值
"""
# 常用数据类型
i2 ：整型int16
i4 ：整型int32
i8 ：整型int64
f4 : 实型单精度single，32位
f8 : 实型双精度double，64位
S1 ：单个字符串 (single-character string)
"""

'\n# 常用数据类型\ni2 ：整型int16\ni4 ：整型int32\ni8 ：整型int64\nf4 : 实型单精度single，32位\nf8 : 实型双精度double，64位\nS1 ：单个字符串 (single-character string)\n'

### 添加变量属性
- unit()：添加变量属性
  - 全局属性：提供一个组或整个数据集的信息
  - 变量属性：提供某个变量的信息（变量属性可以是字符串、数字、序列）
### 检索变量属性的名称
- ncattrs()

In [84]:
import time
f.description = "bogus example script"
f.history = "Created at " + time.ctime(time.time())
f.source = "netCDF4 python module tutorial"
latitudes.units = "degrees north"
longitudes.units = "degrees east"
temp.units = "K"
times.units = "hours since 1900-01-01 00:00:00.0"
times.calendar = "gregorian"

for name in f.ncattrs():
    print("Global attr {} = {}".format(name, getattr(f, name)))

Global attr description = bogus example script
Global attr history = Created at Mon Aug 14 19:28:33 2023
Global attr source = netCDF4 python module tutorial


# NetCDF文件读写
## 写入数据
- 调用numpy,把数据当做一个numpy数组
- 使用numpy数组切片的方式给变量赋值
- 与numPy的数组对象不同的是，如果在当前变量定义的索引范围之外分配数据，具有无限维的NetCDF变量将沿着这些维增长

In [88]:
lats = np.arange(-90, 91, 2.5)
lons = np.arange(-180, 180, 2.5)
latitudes[:] = lats
longitudes[:] = lons

# netCDF 写入数据的维度大小会自动延长
print("temp shape before adding data = {}".format(temp.shape))
from numpy.random import uniform
temp[0:5, 0:10, :, :] = uniform(size=(5, 10, nlats, nlons))
print("temp shape after adding data = {}".format(temp.shape))
print("level shape after adding pressure data = {}".format(level.shape)) 
print(level[:])

temp shape before adding data = (5, 10, 73, 144)
temp shape after adding data = (5, 10, 73, 144)
level shape after adding pressure data = (10,)
[-- -- -- -- -- -- -- -- -- --]


## 读取数据
- NumPy和netCDF变量切片规则之间存在一些差异。
- 切片行为如常，都是start:stop:step三元组。
- numpy使用标量整数索引i获取第i个元素，并将输出数组的秩减少一。
- NetCDF变量，布尔数组和整数序列索引的行为不同于numpy数组。它只允许一维布尔数组和整数序列，这些索引沿着每个维度独立工作138

In [89]:
# 索引沿着每个维度独立工作
print(temp[0, 0, [0,1,2,3], [0,1,2,3]].shape)

(4, 4)


In [91]:
#提取time切片为0、2和4，气压level为850、500和200百帕，所有北半球纬度和东半球经度
tempdat = temp[::2, [1,3,6], lats>0, lons>0]
print("shape of fancy temp slice = {}".format(tempdat.shape))

shape of fancy temp slice = (3, 3, 36, 71)


### 多个NetCDF文件的读取操作
- 使用MFDatset读取数据（MFDataset不能用于写入多文件数据集），使用文件名列表或带通配符的字符串创建MFDataset实例，将文件列表中具有相同无限维度的变量聚集在一起，可以夸多个文件进行切片
- 文件必须采用NETCDF3_64BIT_OFFSET、NETCDF3_64BIT_DATA、NETCDF3_CLASSIC或NETCDF4_CLASSIC格式（MFDataset不支持NETCDF4格式）

In [93]:
# 创建一组具有相同变量名、具有相同无线维度的NetCDF文件
for nf in range(10):
     with nc.Dataset("mftest%s.nc" % nf, "w", format="NETCDF4_CLASSIC") as f:
         _ = f.createDimension("x", None)
         x = f.createVariable("x", "i", ("x",))
         x[0:10] = np.arange(nf*10, 10*(nf+1))

#使用MFDataset一次读取所有文件
from netCDF4 import MFDataset
f = MFDataset("mftest*nc")
print(f.variables["x"][:])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]


# NetCDF的地理参考
## 坐标维度
- 一个NetCDF变量的地理参考与该变量的维度相关联，用于空间定位的维度被称为坐标维度。
- 坐标维度必须是独立变化的维度。
## 坐标变量
- 同时有一个和其名称相同的变量并且该变量的维度就是这个用于空间定位的维度，这个变量被称为坐标变量。
- 坐标变量最常见的用途是在空间和时间上定位数据，但可以为数据变量所依赖的任何其他连续地球物理量（例如密度、温度、辐射波长、天顶辐射角、海面波频率）或离散类别（例如区域类型、模型级别编号、集成成员编号）提供坐标。
- 地理坐标系默认为WGS 84坐标系，根据坐标变量的units属性判断一个变量是不是坐标变量，经度变量和纬度变量的units属性要求不同
    - 表示纬度的变量必须始终显式包含units属性；没有默认值
    - 建议的纬度单位是degrees_north，或者degree_north、degree_N、degrees_N、degreeN和degreesN
    - 表示经度的变量必须始终显式包含units属性；没有默认值。units属性将是根据 udunits.dat 文件格式化的字符串。建议的经度单位为degrees_east，或者degree_east、degree_E、degrees_E、degreeE和degreesE38

  
NetCDF文件标准化的约定（COARDS）对坐标变量进行了定义，但相对而言较为简单。因此，一些学者共同制定并开源了NetCDF气候和预报（CF）元数据公约，旨在促进使用NetCDF应用程序编程器接口 NetCDF 创建的文件的处理和共享。CF公约概括并扩展COARDS公约，其目的是要求符合要求的数据集包含足够的元数据，这些元数据是自描述的，即文件中的每个变量都有对其所表示内容的相关描述，包括物理单位（如果适用），并且每个值可以位于空间（相对于基于地球的坐标）和时间中。

CF公约对四种类型的坐标变量进行了特殊处理：纬度、经度、垂直和时间。CF公约不会根据变量的名称判断一个变量是否是坐标变量，而会根据变量的units和positive属性的值判断。由于这种识别坐标类型的方法很复杂，因此CF公约提供了两种产生直接标识的可选方法。属性axis可以附加到坐标变量，并给定值 X、Y、Z或T之一，分别代表经度、纬度、垂直轴或时间轴。或者，standard_name属性可用于直接标识。但请注意，这些可选属性是对必需的COARDS元数据的补充。

要识别通用空间坐标，需要将axis属性附加到这些坐标变量中，并给定值X、Y或Z之一。轴属性的值X和Y应用于标识水平坐标变量。如果同时标识了X轴和Y轴，则X-Y-up应定义一个右手坐标系，即如果从上方观察，从正X方向到正Y方向的旋转是逆时针的。138

In [ ]:
# 地理坐标系下的纬度变量
float lat(lat) ;
  lat:long_name = "latitude" ;
  lat:units = "degrees_north" ;
  lat:standard_name = "latitude" ;

# 地理坐标系下的经度变量
float lon(lon) ;
  lon:long_name = "longitude" ;
  lon:units = "degrees_east" ;
  lon:standard_name = "longitude" ;



## 绘图
- 参考：https://computing-docs.readthedocs.io/en/latest/cesm/output_cesm.html

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

In [ ]:
data_control = xr.open_dataset("/data/keeling/a/pappup2/a/CESM/cam5_new/CESM_output_data/atm/hist/cam5_new.cam.h0.0020-12.nc") #End of 20 year

#data_control is the experiment run with the control SST.

data_00 = xr.open_dataset("/data/keeling/a/pappup2/a/CESM/cam_00/CESM_output_data/atm/hist/cam_00.cam.h0.0020-12.nc") #End of 20 year

#data_00 is run with a perturbation of 2K at the equator

### Plotting sea surface temperature (Noted as TS)

In [ ]:
data_00.TS

In [ ]:
#surface_t2 = surface_t_2.mean('lon')
surface_t2 = data_control.TS.mean('lon')
surface_t2.plot(color="black",label="Control")
surface_t1 = data_00.TS.mean('lon')
surface_t1.plot(color="red",label=" perturbation")
#plot
plt.legend(loc='lower center')
plt.axvline(x=0,color="black",linestyle='dashed')
plt.axvspan(-10, 10, color='yellow', alpha=0.5)